# Gradient Boosting Regression

In [17]:
import numpy as np
import pandas as pd
import pickle

In [2]:
df = pd.read_csv("./Datasets/clean_immo.csv", sep=',')
df.head()

,zip,type_of_property,subtype_of_property,price,number_of_rooms,house_area,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,commune,province,region,rank
0,1050,house,house,340000,6,203,95,2,0,to be done up,IXELLES,Bruxelles (19 communes),Région Bruxelles-capitale,564.0
1,1880,house,villa,525000,6,250,826,4,0,as new,KAPELLE-OP-DEN-BOS,Brabant Flamand,Région flamande,497.0
2,4900,house,exceptional property,550000,11,475,1543,4,0,good,SPA,Liège,Région wallonne,522.0
3,7912,house,villa,550000,4,325,3570,4,0,good,FRASNES-LEZ-ANVAING,Hainaut,Région wallonne,340.0
4,6032,house,house,550000,5,400,616,3,0,as new,CHARLEROI,Hainaut,Région wallonne,19.0


In [3]:
print(df.isnull().sum())

zip                      0
type_of_property         0
subtype_of_property      0
price                    0
number_of_rooms          0
house_area               0
surface_of_the_land      0
number_of_facades        0
swimming_pool            0
state_of_the_building    0
commune                  0
province                 0
region                   0
rank                     0
dtype: int64


In [4]:
house = df[df['type_of_property'] == 'house']
apart = df[df['type_of_property'] == 'apartment']

In [5]:
hX = house.drop(['zip', 'type_of_property',  'price', 'commune', 'province', 'region', 'number_of_facades'], axis=1)
hy = pd.DataFrame()
hy['price'] = house['price']
print(hX.dtypes)
print(hX.shape)
print(hy.shape)
hX.head()

subtype_of_property       object
number_of_rooms            int64
house_area                 int64
surface_of_the_land        int64
swimming_pool              int64
state_of_the_building     object
rank                     float64
dtype: object
(21603, 7)
(21603, 1)


,subtype_of_property,number_of_rooms,house_area,surface_of_the_land,swimming_pool,state_of_the_building,rank
0,house,6,203,95,0,to be done up,564.0
1,villa,6,250,826,0,as new,497.0
2,exceptional property,11,475,1543,0,good,522.0
3,villa,4,325,3570,0,good,340.0
4,house,5,400,616,0,as new,19.0


In [6]:
aX = apart.drop(['zip', 'type_of_property',  'price', 'commune', 'province', 'region'], axis=1)
ay = pd.DataFrame()
ay['price'] = apart['price']
print(aX.dtypes)
print(aX.shape)
print(ay.shape)
aX.head()

subtype_of_property       object
number_of_rooms            int64
house_area                 int64
surface_of_the_land        int64
number_of_facades          int64
swimming_pool              int64
state_of_the_building     object
rank                     float64
dtype: object
(19824, 8)
(19824, 1)


,subtype_of_property,number_of_rooms,house_area,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,rank
19825,ground floor,1,102,166,3,0,as new,391.0
19826,flat studio,1,30,5036,1,0,good,118.0
19827,flat studio,1,30,5036,1,0,good,118.0
19828,apartment,2,60,310,2,0,as new,70.0
19829,apartment,1,48,128,3,0,just renovated,40.0


In [7]:
# Get dummies for object type
X_columns_name = ['subtype_of_property', 'state_of_the_building']
hX = pd.get_dummies(hX, columns=X_columns_name, prefix=X_columns_name, drop_first=True)
aX = pd.get_dummies(aX, columns=X_columns_name, prefix=X_columns_name, drop_first=True)

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [9]:
hX_train, hX_test, hy_train, hy_test = train_test_split(hX, hy, test_size=0.2)
aX_train, aX_test, ay_train, ay_test = train_test_split(aX, ay, test_size=0.2)

In [10]:
hest = GradientBoostingRegressor(n_estimators=1000, max_depth=4, min_samples_split=2, learning_rate=0.1, loss='ls')
aest = GradientBoostingRegressor(n_estimators=1000, max_depth=4, min_samples_split=2, learning_rate=0.1, loss='ls')

In [11]:
hest.fit(hX_train, hy_train)

/home/demes/dev/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingRegressor(max_depth=4, n_estimators=1000)

In [12]:
aest.fit(aX_train, ay_train)

/home/demes/dev/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingRegressor(max_depth=4, n_estimators=1000)

In [13]:
print(f"Score training House : {hest.score(hX_train, hy_train)}")
print(f"Score training Apart : {aest.score(aX_train, ay_train)}")

Score training House : 0.8721587492583411
Score training Apart : 0.8407730779307127


In [14]:
hy_pred = hest.predict(hX_test)
ay_pred = aest.predict(aX_test)

In [15]:
print(f"Score test House : {hest.score(hX_test, hy_test)}")
print(f"Score test Apart : {aest.score(aX_test, ay_test)}")

Score test House : 0.7763433841042303
Score test Apart : 0.751338770436632


In [19]:
hfilename = 'house_model.sav'
pickle.dump(hest, open(hfilename, 'wb'))

In [21]:
afilename = 'apart_model.sav'
pickle.dump(aest, open(afilename, 'wb'))

In [23]:
hX

,number_of_rooms,house_area,surface_of_the_land,swimming_pool,rank,subtype_of_property_chalet,subtype_of_property_country cottage,subtype_of_property_exceptional property,subtype_of_property_farmhouse,subtype_of_property_house,...,subtype_of_property_mixed use building,subtype_of_property_other property,subtype_of_property_town house,subtype_of_property_villa,state_of_the_building_good,state_of_the_building_just renovated,state_of_the_building_to be done up,state_of_the_building_to renovate,state_of_the_building_to restore,state_of_the_building_unknown
0,6,203,95,0,564.0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,6,250,826,0,497.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,11,475,1543,0,522.0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,325,3570,0,340.0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,5,400,616,0,19.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41422,2,152,152,0,448.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
41423,3,155,155,0,416.0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
41424,0,1007,1007,0,230.0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
41425,5,139,139,0,334.0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [24]:
aX

,number_of_rooms,house_area,surface_of_the_land,number_of_facades,swimming_pool,rank,subtype_of_property_duplex,subtype_of_property_flat studio,subtype_of_property_ground floor,subtype_of_property_kot,subtype_of_property_loft,subtype_of_property_penthouse,subtype_of_property_service flat,subtype_of_property_triplex,state_of_the_building_good,state_of_the_building_just renovated,state_of_the_building_to be done up,state_of_the_building_to renovate,state_of_the_building_to restore,state_of_the_building_unknown
19825,1,102,166,3,0,391.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
19826,1,30,5036,1,0,118.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
19827,1,30,5036,1,0,118.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
19828,2,60,310,2,0,70.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19829,1,48,128,3,0,40.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40101,2,157,157,2,0,570.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
40102,3,164,164,3,0,201.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
40103,2,111,111,3,0,194.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
40104,2,182,182,3,0,467.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [28]:
hest.predict([[3, 154, 543, 0, 322, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]])[0]

321818.8082331986